# Workflow

## Collect Parameters and Providers
### Simulation(McStas) Data
There is a dedicated loader, ``load_mcstas_nexus`` for ``McStas`` simulation data workflow. <br>
``MaximumProbability`` can be manually provided to the loader <br>
to derive more realistic number of events. <br>
It is because ``weights`` are given as probability, not number of events in a McStas file. <br>

In [ ]:
# Collect parameters and providers
import scipp as sc
from ess.nmx.mcstas_loader import load_mcstas_nexus
from ess.nmx.mcstas_loader import (
    InputFilepath,
    MaximumProbability,
    DefaultMaximumProbability
)
from ess.nmx.data import small_mcstas_sample
from ess.nmx.reduction import bin_time_of_arrival, NMXReducedData, TimeBinSteps

providers = (load_mcstas_nexus, bin_time_of_arrival, )

file_path = small_mcstas_sample()  # Replace it with your data file path
params = {
    TimeBinSteps: TimeBinSteps(50),
    InputFilepath: InputFilepath(file_path),
    # Additional parameters for McStas data handling.
    MaximumProbability: DefaultMaximumProbability,
}

In [ ]:
from typing import get_type_hints
param_reprs = {key.__name__: value for key, value in params.items()}
prov_reprs = {
    get_type_hints(prov)['return'].__name__: prov.__name__ for prov in providers
}

# Providers and parameters to be used for pipeline
sc.DataGroup(**prov_reprs, **param_reprs)

## Build Workflow

In [ ]:
import sciline as sl
from ess.nmx.mcstas_loader import NMXData
from ess.nmx.reduction import NMXReducedData

nmx_pl = sl.Pipeline(list(providers), params=params)
nmx_workflow = nmx_pl.get(NMXReducedData)
nmx_workflow.visualize()

## Compute Desired Types

In [ ]:
# Event data grouped by detector panel and pixel id.
dg = nmx_workflow.compute(NMXData)
dg

In [ ]:
# Binned data.

binned_dg = nmx_workflow.compute(NMXReducedData)
binned_dg

## Export Results

``NMXReducedData`` object has a method to export the data into nexus or h5 file.

You can save the result as ``test.nxs`` for example.

```python
binned_dg.export_as_nexus('test.nxs')
```

## Instrument View

Pixel positions are not used for later steps,
but it is included in the coordinates for instrument view.

All pixel positions are relative to the sample position,
therefore the sample is at (0, 0, 0).

The instrument view is shown using

In [ ]:
import scippneutron as scn

da = dg['weights']
da.coords['position'] = dg['position']
# Plot one out of 100 pixels to reduce size of docs output
view = scn.instrument_view(da.flatten(['panel', 'id'], 'id')['id', ::100].hist(), pixel_size=0.0075)
view.children[0].toolbar.cameraz()
view
```

**It might be very slow or not work in the ``VS Code`` jupyter notebook editor.**